<a href="https://colab.research.google.com/github/mancinimassimiliano/DeepLearningLab/blob/master/myFirstNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision

DEVICE='cuda:0'
LR=0.1
WEIGHT_DECAY = 0.000001 
MOMENTUM = 0.9
BATCH_SIZE = 256
TEST_BATCH_SIZE = 500
EPOCHS = 20

INPUT_DIM = 784
HIDDEN_DIM = 100
OUTPUT_DIM = 10


class MyFirstNetwork(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim):
    super(MyFirstNetwork, self).__init__()
    # TODO: Initialize layers

  def forward(self, x):
    # TODO: Define forward pass
    return x
  

def get_data(): 
  # This function is needed to convert the PIL images to Tensors
  transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

  # Load MNIST data
  full_training_data = None # TODO 
  test_data = None # TODO
  

  # Create train test splits
  num_samples = len(full_training_data)
  training_samples = int(num_samples*0.5+1)
  validation_samples = num_samples - training_samples

  training_data, validation_data = torch.utils.data.random_split(full_training_data, [training_samples, validation_samples])

  # Initialize dataloaders
  train_loader = None # TODO
  val_loader = None # TODO
  test_loader = None # TODO
  
  return train_loader, val_loader, test_loader


def get_optimizer(net, lr, wd, momentum):
  # TODO: Create and optimizer (hint: have a look at torch.optim; you can retrieve networks params through net.parameters())
  return None


def get_cost_function():
  # TODO: Initialize the loss function (hint: have a look at torch.nn)
  return None



def test(net, data_loader, cost_function):
  net.eval()
  samples = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.

  with torch.no_grad():
    for batch_idx, (inputs, targets) in enumerate(data_loader):
      # Load data into GPU
        
      # Forward pass
      outputs = None

      # Apply the loss
      loss = None

      # Better print something, no?
      samples+=inputs.shape[0]
      cumulative_loss += loss.item()
      _, predicted = outputs.max(1)
      cumulative_accuracy += predicted.eq(targets).sum().item()

  return cumulative_loss/samples, cumulative_accuracy/samples*100




def train(net,data_loader,optimizer,cost_function):
  net.train()
  samples = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.

  for batch_idx, (inputs, targets) in enumerate(data_loader):
    # Load data into GPU
      
    # Forward pass
    outputs = None

    # Apply the loss
    loss = None

    # Reset the optimizer
      
    # Backward pass
      
    # Update parameters

    # Better print something, no?
    samples+=inputs.shape[0]
    cumulative_loss += loss.item()
    _, predicted = outputs.max(1)
    cumulative_accuracy += predicted.eq(targets).sum().item()

  return cumulative_loss/samples, cumulative_accuracy/samples*100


def main():
  train_loader, val_loader, test_loader = get_data()
  net = MyFirstNetwork(INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM).to(DEVICE)
  optimizer = get_optimizer(net, LR, WEIGHT_DECAY, MOMENTUM)
  cost_function = get_cost_function()

  print('Before training:')
  train_loss, train_accuracy = test(net, train_loader, cost_function)
  val_loss, val_accuracy = test(net, val_loader, cost_function)
  test_loss, test_accuracy = test(net, test_loader, cost_function)

  print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
  print('\t Validation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
  print('\t Test loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
  print('-----------------------------------------------------')

  for e in range(EPOCHS):
    train_loss, train_accuracy = train(net, train_loader, optimizer, cost_function)
    val_loss, val_accuracy = test(net, val_loader, cost_function)
    print('Epoch: {:d}'.format(e+1))
    print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
    print('\t Validation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
    print('-----------------------------------------------------')

  print('After training:')
  train_loss, train_accuracy = test(net, train_loader, cost_function)
  val_loss, val_accuracy = test(net, val_loader, cost_function)
  test_loss, test_accuracy = test(net, test_loader, cost_function)

  print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
  print('\t Validation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
  print('\t Test loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
  print('-----------------------------------------------------')
